In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers langchain llama-index llama-index-embeddings-huggingface
!pip install -q pypdf sentence_transformers accelerate bitsandbytes
!pip install llama-index transformers torch accelerate bitsandbytes
!pip install --upgrade llama-index llama-index-llms-huggingface transformers torch accelerate bitsandbytes



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [3]:
# Import required libraries
from llama_index.readers.file import PDFReader

# Load the PDF
pdf_path = "/content/drive/MyDrive/Colab Notebooks/edit2016.pdf"
documents = PDFReader().load_data(file=pdf_path)

# Check if the document loaded correctly
print(documents[0].text[:500])  # Print first 500 characters


 
BIO-MEDICAL WASTE MANAGEMENT RULES, 2016 as amended till 2019  
[Published in the Gazette of India, Extraordinary, Part II, Section 3, Sub-section (i)]  
 GOVERNMENT OF INDIA  
MINISTRY OF ENVIRONMENT, FOREST AND CLIMATE CHANGE  
NOTIFICATION  
New Delhi, the 28th March, 2016  
G.S.R. 343(E).-Whereas the Bio-Medical Waste (Management and Handling) Rules, 1998 was published 
vide notification number S.O. 630 (E) dated the 20 th July, 1998, by the Government of India in the 
erstwhile Ministry o


In [4]:
system_prompt = """
You are MedWaste Guardian, an AI assistant specializing in biomedical waste compliance.
Your goal is to provide accurate legal guidance based on the Bio-Medical Waste Management Rules, 2016.
Use retrieved legal texts to generate well-structured and compliant responses.
If information is unavailable, state so clearly.
"""
from llama_index.core.prompts.prompts import SimpleInputPrompt

# Define query wrapper prompt
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")


In [5]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `Llama` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [6]:
# Install necessary dependencies (if not installed)
!pip install --upgrade llama-index llama-index-llms-huggingface transformers torch accelerate bitsandbytes

from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig
import torch

# Configure quantization to optimize memory usage
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # Offload some layers to CPU if needed
)

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
# Initialize the LLaMA-2 model
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 1.0, "do_sample": False},  # ✅ Fixed temperature issue
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-3.2-1B-Instruct",
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto",  # Auto-assigns layers across GPU/CPU
    model_kwargs={"torch_dtype": torch.float16, "quantization_config": bnb_config}
)

print("LLaMA-2 model initialized successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

LLaMA-2 model initialized successfully!


In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Initialize embedding model and ensure it runs on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", device=device)

print("Embedding model loaded successfully on:", device)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded successfully on: cuda


In [8]:
from huggingface_hub import login
from transformers import AutoTokenizer
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM


# ✅ Load tokenizer for LLaMA-2
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)  # Updated argument

# ✅ Ensure LlamaIndex doesn't default to OpenAI
Settings.llm = None

# ✅ Load LLaMA-2 model
llm = HuggingFaceLLM(model_name=model_name, tokenizer=tokenizer)

# ✅ Set LLaMA-2 as the LLM in LlamaIndex
Settings.llm = llm

print("✅ LLaMA-2 is now the active LLM:", Settings.llm.model_name)  # Avoid recursion error


LLM is explicitly disabled. Using MockLLM.
✅ LLaMA-2 is now the active LLM: meta-llama/Llama-3.2-1B-Instruct


In [ ]:
from llama_index.readers.file import PDFReader
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#Use local embeddings instead of OpenAI
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embed_model  # Ensure LlamaIndex uses this embedding model

#Load PDF
pdf_path = "/content/drive/MyDrive/Colab Notebooks/edit2016.pdf"
documents = PDFReader().load_data(file=pdf_path)

#Create the index
index = VectorStoreIndex.from_documents(documents, settings=Settings)

print("Index created successfully!")

✅ Index created successfully!


In [ ]:
# Create a query engine from the index
query_engine = index.as_query_engine(llm=llm)

In [ ]:
# Example query
response = query_engine.query("How to dispose of syringes?")

# Print response
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Syringes should be either mutilated or needles should be cut and or stored in puncture proof, leak proof and tamper proof containers for sharps storage. Wherever the occupier is not linked to a disposal facility it shall be the responsibility of the occupier to sterilize and dispose in the manner prescribed.

Answer: 
1. Syringes should be either mutilated or needles should be cut and or stored in puncture proof, leak proof and tamper proof containers for sharps storage. 
2. Wherever the occupier is not linked to a disposal facility it shall be the responsibility of the occupier to sterilize and dispose in the manner prescribed.


In [ ]:
index.storage_context.persist(persist_dir="./medwaste_index")

In [13]:
!zip -r /content/medwaste_guardian_rag.zip /content/drive/MyDrive/Colab\ Notebooks/medwaste_index


  adding: content/drive/MyDrive/Colab Notebooks/medwaste_index/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/medwaste_index/graph_store.json (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/medwaste_index/image__vector_store.json (deflated 19%)
  adding: content/drive/MyDrive/Colab Notebooks/medwaste_index/index_store.json (deflated 64%)
  adding: content/drive/MyDrive/Colab Notebooks/medwaste_index/docstore.json (deflated 72%)
  adding: content/drive/MyDrive/Colab Notebooks/medwaste_index/default__vector_store.json (deflated 57%)


In [ ]:
from google.colab import files
files.download("/content/medwaste_guardian_rag.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
#pip install pypdf
#pip install -q transformers einops accelerate langchain bitsandbytes
#pip install llama_index
#pip install --upgrade llama-index llama-index-llms-huggingface transformers
#pip install llama-index llama-index-readers-file
#pip install sentence_transformers
#pip install -U llama-index-embeddings-huggingface langchain langchain-community
#pip install fastapi uvicorn


In [16]:
response = query_engine.query("What are the biomedical waste disposal rules for human anatomical waste?")
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 The biomedical waste disposal rules for human anatomical waste include the following:
1.  The disposal of human anatomical waste should be carried out in a bio-medical waste treatment and disposal facility. The facility should have the necessary equipment and infrastructure to handle the waste safely and efficiently.
2.  The waste should be segregated and labeled correctly before disposal.
3.  The disposal facility should have the necessary permits and approvals to handle the waste.
4.  The disposal facility should have a system for tracking and monitoring the disposal of the waste.
5.  The disposal facility should have a system for disposing of any hazardous materials present in the waste.
6.  The disposal facility should have a system for providing information to the public about the disposal of the waste.
7.  The disposal facility should have a system for conducting regular audits and inspections to ensure compliance with the rules.
8.  The disposal facility should have a system fo